In [1]:
#PARTE 1
#INCISO 2
import pandas as pd

# Rutas de los archivos
ruta_hogar = 'C:/Users/User/Downloads/Big data - TP2/TP3/usu_hogar_T423.xlsx'
ruta_individual = 'C:/Users/User/Downloads/Big data - TP2/TP3/usu_individual_T423.xlsx'


# Lectura de los archivos de Excel
datos_hogar = pd.read_excel(ruta_hogar)
datos_individual = pd.read_excel(ruta_individual)

# Renombrar columnas para evitar conflictos en el merge
datos_individual.rename(columns={'ITF': 'ITF_persona', 'IPCF': 'IPCF_persona'}, inplace=True)

# Filtrar datos para GBA (REGION == 1)
gba_hogar = datos_hogar[datos_hogar['REGION'] == 1]
gba_individual = datos_individual[datos_individual['REGION'] == 1]

# Fusionar los DataFrames filtrados
datos_combinados = pd.merge(gba_individual, gba_hogar, on=['CODUSU', 'NRO_HOGAR'])

# Mostrar primeras filas del DataFrame combinado
print("Datos Combinados:\n", datos_combinados.head())

Datos Combinados:
                           CODUSU  ANO4_x  TRIMESTRE_x  NRO_HOGAR  COMPONENTE  \
0  TQRMNOTUUHJMLPCDEIIAD00801670    2023            4          1           1   
1  TQRMNOTUUHJMLPCDEIIAD00801670    2023            4          1           2   
2  TQRMNOPUUHJKLQCDEIIAD00793187    2023            4          1           1   
3  TQRMNOPUUHJKLQCDEIIAD00793187    2023            4          1           2   
4  TQRMNOPUTHKLMNCDEIIAD00791268    2023            4          1           1   

   H15  REGION_x MAS_500_x  AGLOMERADO_x  PONDERA_x  ...  GDECCFR_y  \
0    1         1         S            32       1775  ...        8.0   
1    1         1         S            32       1775  ...        8.0   
2    2         1         S            32        609  ...       12.0   
3    1         1         S            32        609  ...       12.0   
4    1         1         S            32       3129  ...        8.0   

   PDECCFR_y ADECCFR_y  PONDIH_y  VII1_1  VII1_2  VII2_1  VII2_2  VII2_3 

In [2]:
#INCISO 3:
# Filtrar variables binarias para valores válidos (1 o 2)
variables_binarias = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19_A', 'V19_B', 'V21', 'V22']
for var in variables_binarias:
    if var in datos_combinados.columns:
        datos_combinados = datos_combinados[datos_combinados[var].isin([1, 2])]

# Eliminar columnas que no aportan valor
columnas_eliminar = ['IV1_ESP', 'IV3_ESP', 'IV7_ESP', 'II7_ESP', 'II8_ESP']
datos_combinados.drop(columns=[col for col in columnas_eliminar if col in datos_combinados.columns], inplace=True)

# Filtrar ingresos, edad, horas trabajadas, cantidad de habitantes en el hogar y habitaciones
if 'IPCF' in datos_combinados.columns and 'ITF_persona' in datos_combinados.columns:
    datos_combinados = datos_combinados[(datos_combinados['IPCF'] >= 0) & (datos_combinados['ITF_persona'] >= 0)]
if 'CH06' in datos_combinados.columns:
    datos_combinados = datos_combinados[datos_combinados['CH06'] >= 0]
if 'PP3E_TOT' in datos_combinados.columns:
    datos_combinados = datos_combinados[(datos_combinados['PP3E_TOT'] >= 0) & (datos_combinados['PP3E_TOT'] <= 168)]
if 'PP3F_TOT' in datos_combinados.columns:
    datos_combinados = datos_combinados[(datos_combinados['PP3F_TOT'] >= 0) & (datos_combinados['PP3F_TOT'] <= 168)]
if 'IX_TOT' in datos_combinados.columns and 'IX_MEN10' in datos_combinados.columns and 'IX_MAYEQ10' in datos_combinados.columns:
    datos_combinados = datos_combinados[(datos_combinados['IX_TOT'] >= 0) & (datos_combinados['IX_MEN10'] >= 0) & (datos_combinados['IX_MAYEQ10'] >= 0)]
if 'II1' in datos_combinados.columns and 'II2' in datos_combinados.columns:
    datos_combinados = datos_combinados[(datos_combinados['II1'] >= 0) & (datos_combinados['II2'] >= 0) & (datos_combinados['II2'] <= datos_combinados['II1'])]
if 'II3_1' in datos_combinados.columns and 'II3' in datos_combinados.columns:
    datos_combinados = datos_combinados[(datos_combinados['II3_1'] >= 0) & (datos_combinados['II3_1'] <= datos_combinados['II3'])]

# Mostrar el DataFrame después de filtrar los valores sin sentido
print("\nDataFrame Limpio:")
print(datos_combinados.head())


DataFrame Limpio:
                           CODUSU  ANO4_x  TRIMESTRE_x  NRO_HOGAR  COMPONENTE  \
0   TQRMNOTUUHJMLPCDEIIAD00801670    2023            4          1           1   
4   TQRMNOPUTHKLMNCDEIIAD00791268    2023            4          1           1   
5   TQRMNOPUTHKLMNCDEIIAD00791268    2023            4          1           2   
7   TQRMNORXPHKLMNCDEIIAD00791273    2023            4          2           1   
11  TQRMNOQWYHLMKRCDEIJAH00854987    2023            4          1           1   

    H15  REGION_x MAS_500_x  AGLOMERADO_x  PONDERA_x  ...  GDECCFR_y  \
0     1         1         S            32       1775  ...        8.0   
4     1         1         S            32       3129  ...        8.0   
5     1         1         S            32       3129  ...        8.0   
7     1         1         S            32       3357  ...        8.0   
11    1         1         S            33       1656  ...        4.0   

    PDECCFR_y ADECCFR_y  PONDIH_y  VII1_1  VII1_2  VII2_1  VI

In [4]:
#INCISO 4 y 5 :
# Crear nuevas variables relevantes para predecir pobreza
datos_combinados['habitantes_por_cuarto'] = datos_combinados['IX_TOT'] / datos_combinados['II2']
datos_combinados['ingreso_por_adulto'] = datos_combinados['ITF_persona'] / datos_combinados['IX_MAYEQ10']
datos_combinados['asistencia_familiar'] = datos_combinados.groupby(['CODUSU', 'NRO_HOGAR'])['V5_M'].transform('sum')

# Seleccionar variables relevantes para el análisis
variables_relevantes = ['habitantes_por_cuarto', 'ingreso_por_adulto', 'IPCF', 'PP3E_TOT', 'asistencia_familiar']

# Calcular estadísticas descriptivas
estadisticas_descriptivas = datos_combinados[variables_relevantes].describe()

# Mostrar estadísticas descriptivas
print("\nEstadísticas Descriptivas:")
print(estadisticas_descriptivas)


Estadísticas Descriptivas:
       habitantes_por_cuarto  ingreso_por_adulto          IPCF     PP3E_TOT  \
count            3443.000000        3.443000e+03  3.443000e+03  3443.000000   
mean                     inf        1.453690e+05  1.314657e+05    36.579437   
std                      NaN        2.101405e+05  1.960944e+05    17.548398   
min                 0.333333        0.000000e+00  0.000000e+00     0.000000   
25%                 1.333333        0.000000e+00  0.000000e+00    24.000000   
50%                 1.500000        8.666667e+04  7.333333e+04    40.000000   
75%                 2.000000        2.150667e+05  2.000000e+05    48.000000   
max                      inf        2.100000e+06  2.100000e+06   133.000000   

       asistencia_familiar  
count          3443.000000  
mean           4008.039791  
std           15738.100568  
min              -9.000000  
25%               0.000000  
50%               0.000000  
75%               0.000000  
max          166000.000000  

In [2]:
#INCISO 6:
import matplotlib.pyplot as plt
# Seleccionar las variables para el gráfico
x_variable = datos_combinados['ITF_persona']
y_variable = datos_combinados['V5']

# Crear un gráfico de dispersión
plt.figure(figsize=(10, 6))
plt.scatter(x_variable, y_variable, alpha=0.5)
plt.xlabel('Ingreso Total Familiar (ITF)')
plt.ylabel('Recepción de Ayuda Social (V5)')
plt.title('Relación entre ITF y Recepción de Ayuda Social (V5)')

# Ajustar el rango del eje x
plt.xlim(left=0, right=5000000)

# Añadir grid
plt.grid(True)

# Mostrar el gráfico
plt.show()

# Añadir comentario al gráfico
plt.text(50, 1.5,
         "Observamos que los hogares con mayores niveles de ITF tienden a no recibir ayudas sociales.",
         fontsize=10, style='italic',
         bbox={'facecolor': 'lightblue', 'alpha': 0.5, 'pad': 10})

plt.show()


NameError: name 'datos_combinados' is not defined

In [6]:
#INCISO 7:
#Volvemos a correr ejercicios del TP3.
import pandas as pd

# Leer los archivos Excel
usu_individual = pd.read_excel('C:/Users/User/Downloads/Big data - TP2/TP3/usu_individual_T423.xlsx')
tabla_adulto_equiv = pd.read_excel('C:/Users/User/Downloads/Big data - TP2/TP3/tabla_adulto_equiv2.xlsx')

# Eliminar los espacios adicionales en los nombres de las columnas
usu_individual.columns = usu_individual.columns.str.strip()

# Crear un diccionario para mapear los valores de adulto equivalente
equivalencias = {
    (row['Edad'], 'Mujeres'): row['Mujeres'] for idx, row in tabla_adulto_equiv.iterrows()
}
equivalencias.update({
    (row['Edad'], 'Varones'): row['Varones'] for idx, row in tabla_adulto_equiv.iterrows()
})

# Convertir la columna de edad en `usu_individual` para coincidir con el formato de `tabla_adulto_equiv`
usu_individual['Edad'] = usu_individual['CH06'].apply(lambda x: f"{x} años" if x > 0 else "Menor de 1 año")

# Asignar el valor de adulto equivalente con valores por defecto
def obtener_equivalente(row):
    if (row['Edad'], 'Mujeres' if row['CH04'] == 2 else 'Varones') in equivalencias:
        return equivalencias[(row['Edad'], 'Mujeres' if row['CH04'] == 2 else 'Varones')]
    else:
        return 1.0 if row['CH06'] > 18 else 0.5

usu_individual['adulto_equiv'] = usu_individual.apply(obtener_equivalente, axis=1)

# Sumar los valores de adulto_equiv para cada hogar
ad_equiv_hogar = usu_individual.groupby(['CODUSU', 'NRO_HOGAR'])['adulto_equiv'].sum().reset_index()

# Renombrar la columna
ad_equiv_hogar.rename(columns={'adulto_equiv': 'ad_equiv_hogar'}, inplace=True)

# Unir esta nueva columna con la tabla original `usu_individual`
usu_individual = usu_individual.merge(ad_equiv_hogar, on=['CODUSU', 'NRO_HOGAR'], how='left')

# Verificar los nombres de las columnas después de la unión
print(usu_individual.columns)

# Limpiar la duplicación de columnas si es necesario
if 'ad_equiv_hogar_x' in usu_individual.columns and 'ad_equiv_hogar_y' in usu_individual.columns:
    usu_individual = usu_individual.drop(columns=['ad_equiv_hogar_x']).rename(columns={'ad_equiv_hogar_y': 'ad_equiv_hogar'})

# Mostrar las primeras filas del DataFrame resultante
print(usu_individual[['CODUSU', 'NRO_HOGAR', 'CH04', 'CH06', 'Edad', 'adulto_equiv', 'ad_equiv_hogar']].head(20))

Index(['CODUSU', 'ANO4', 'TRIMESTRE', 'NRO_HOGAR', 'COMPONENTE', 'H15',
       'REGION', 'MAS_500', 'AGLOMERADO', 'PONDERA',
       ...
       'DECCFR', 'IDECCFR', 'RDECCFR', 'GDECCFR', 'PDECCFR', 'ADECCFR',
       'PONDIH', 'Edad', 'adulto_equiv', 'ad_equiv_hogar'],
      dtype='object', length=180)
                           CODUSU  NRO_HOGAR  CH04  CH06     Edad  \
0   TQRMNOTUUHJMLPCDEIIAD00801670          1     1    76  76 años   
1   TQRMNOTUUHJMLPCDEIIAD00801670          1     2    79  79 años   
2   TQRMNOPUUHJKLQCDEIIAD00793187          1     1    65  65 años   
3   TQRMNOPUUHJKLQCDEIIAD00793187          1     2    66  66 años   
4   TQRMNOPUTHKLMNCDEIIAD00791268          1     2    49  49 años   
5   TQRMNOPUTHKLMNCDEIIAD00791268          1     1    20  20 años   
6   TQRMNOPUTHKLMNCDEIIAD00791268          1     1    17  17 años   
7   TQRMNORXPHKLMNCDEIIAD00791273          2     1    39  39 años   
8   TQRMNORXPHKLMNCDEIIAD00791273          2     1     5   5 años   
9   TQRM

In [7]:
# 3.
# Cuantas personas no respondieron cuál es su ingreso total familiar (ITF)
norespondieron = usu_individual[usu_individual['ITF'].isna() | (usu_individual['ITF'] == 0)]
respondieron = usu_individual[~usu_individual['ITF'].isna() & (usu_individual['ITF'] != 0)]

# Guardar las bases distintas
norespondieron.to_csv('norespondieron.csv', index=False)
respondieron.to_csv('respondieron.csv', index=False)

# Mostrar el número de personas que no respondieron
print(f"Cantidad de personas que no respondieron su ITF: {len(norespondieron)}")
print(f"Cantidad de personas que respondieron su ITF: {len(respondieron)}")


Cantidad de personas que no respondieron su ITF: 2979
Cantidad de personas que respondieron su ITF: 4401


In [8]:
# 4. 
import pandas as pd

# Cargar la base respondieron
respondieron = pd.read_csv('respondieron.csv', low_memory=False)

# Definir el valor de la Canasta Básica Total para un adulto equivalente
canasta_basica_total = 132853.3

# Calcular el ingreso necesario para cada hogar
respondieron['ingreso_necesario'] = respondieron['ad_equiv_hogar'] * canasta_basica_total

# Guardar la base actualizada
respondieron.to_csv('respondieron_actualizado.csv', index=False)

# Mostrar las primeras filas del DataFrame resultante para verificar
print(respondieron[['CODUSU', 'NRO_HOGAR', 'ad_equiv_hogar', 'ingreso_necesario']].head(20))

                           CODUSU  NRO_HOGAR  ad_equiv_hogar  \
0   TQRMNOTUUHJMLPCDEIIAD00801670          1            2.00   
1   TQRMNOTUUHJMLPCDEIIAD00801670          1            2.00   
2   TQRMNOPUTHKLMNCDEIIAD00791268          1            3.04   
3   TQRMNOPUTHKLMNCDEIIAD00791268          1            3.04   
4   TQRMNOPUTHKLMNCDEIIAD00791268          1            3.04   
5   TQRMNORXPHKLMNCDEIIAD00791273          2            1.60   
6   TQRMNORXPHKLMNCDEIIAD00791273          2            1.60   
7   TQRMNOSPTHKKMPCDEIIAD00791441          1            2.00   
8   TQRMNOSPTHKKMPCDEIIAD00791441          1            2.00   
9   TQRMNOQWYHLMKRCDEIJAH00854987          1            2.00   
10  TQRMNOQWYHLMKRCDEIJAH00854987          1            2.00   
11  TQRMNOPWXHJMLQCDEIJAH00794277          1            5.01   
12  TQRMNOPWXHJMLQCDEIJAH00794277          1            5.01   
13  TQRMNOPWXHJMLQCDEIJAH00794277          1            5.01   
14  TQRMNOPWXHJMLQCDEIJAH00794277       

In [9]:
#INCISO 8 Y 9
import pandas as pd

# Leer los archivos Excel
usu_individual = pd.read_excel('C:/Users/User/Downloads/Big data - TP2/TP3/usu_individual_T423.xlsx')
tabla_adulto_equiv = pd.read_excel('C:/Users/User/Downloads/Big data - TP2/TP3/tabla_adulto_equiv2.xlsx')

# Eliminar los espacios adicionales en los nombres de las columnas
usu_individual.columns = usu_individual.columns.str.strip()

# Crear un diccionario para mapear los valores de adulto equivalente
equivalencias = {
    (row['Edad'], 'Mujeres'): row['Mujeres'] for idx, row in tabla_adulto_equiv.iterrows()
}
equivalencias.update({
    (row['Edad'], 'Varones'): row['Varones'] for idx, row in tabla_adulto_equiv.iterrows()
})

# Convertir la columna de edad en `usu_individual` para coincidir con el formato de `tabla_adulto_equiv`
usu_individual['Edad'] = usu_individual['CH06'].apply(lambda x: f"{x} años" if x > 0 else "Menor de 1 año")

# Asignar el valor de adulto equivalente con valores por defecto
def obtener_equivalente(row):
    if (row['Edad'], 'Mujeres' if row['CH04'] == 2 else 'Varones') in equivalencias:
        return equivalencias[(row['Edad'], 'Mujeres' if row['CH04'] == 2 else 'Varones')]
    else:
        return 1.0 if row['CH06'] > 18 else 0.5

usu_individual['adulto_equiv'] = usu_individual.apply(obtener_equivalente, axis=1)

# Sumar los valores de adulto_equiv para cada hogar
ad_equiv_hogar = usu_individual.groupby(['CODUSU', 'NRO_HOGAR'])['adulto_equiv'].sum().reset_index()

# Renombrar la columna
ad_equiv_hogar.rename(columns={'adulto_equiv': 'ad_equiv_hogar'}, inplace=True)

# Unir esta nueva columna con la tabla original `usu_individual`
usu_individual = usu_individual.merge(ad_equiv_hogar, on=['CODUSU', 'NRO_HOGAR'], how='left')

# Limpiar la duplicación de columnas si es necesario
if 'ad_equiv_hogar_x' in usu_individual.columns and 'ad_equiv_hogar_y' in usu_individual.columns:
    usu_individual = usu_individual.drop(columns=['ad_equiv_hogar_x']).rename(columns={'ad_equiv_hogar_y': 'ad_equiv_hogar'})

print(usu_individual[['CODUSU', 'NRO_HOGAR', 'CH04', 'CH06', 'Edad', 'adulto_equiv', 'ad_equiv_hogar']].head(20))

# Dividir los datos en respondieron y norespondieron según ITF_persona
respondieron = usu_individual[usu_individual['ITF'] > 0].copy()
norespondieron = usu_individual[usu_individual['ITF'] == 0].copy()

# Guardar las bases distintas
respondieron.to_csv('C:/Users/User/Downloads/Big data - TP2/TP3/respondieron.csv', index=False)
norespondieron.to_csv('C:/Users/User/Downloads/Big data - TP2/TP3/norespondieron.csv', index=False)

print(f"Cantidad de personas que no respondieron su ITF: {len(norespondieron)}")
print(f"Cantidad de personas que respondieron su ITF: {len(respondieron)}")

# Definir el valor de la Canasta Básica Total para un adulto equivalente
canasta_basica_total = 132853.3

# Calcular el ingreso necesario para cada hogar en respondieron
respondieron['ingreso_necesario'] = respondieron['ad_equiv_hogar'] * canasta_basica_total

# Calcular si la familia es pobre en respondieron
respondieron['pobre'] = (respondieron['ITF'] < respondieron['ingreso_necesario']).astype(int)

# Guardar la base actualizada
respondieron.to_csv('C:/Users/User/Downloads/Big data - TP2/TP3/respondieron_actualizado.csv', index=False)

# Mostrar las primeras filas del DataFrame resultante para verificar
print(respondieron[['CODUSU', 'NRO_HOGAR', 'ITF', 'ad_equiv_hogar', 'ingreso_necesario', 'pobre']].head(20))

# Mostrar el número total de hogares pobres
total_hogares_pobres = respondieron[respondieron['pobre'] == 1].shape[0]
total_hogares = respondieron.shape[0]

print(f"Total de hogares pobres: {total_hogares_pobres}")
print(f"Total de hogares: {total_hogares}")
print(f"Proporción de hogares pobres: {total_hogares_pobres / total_hogares:.2%}")

# Guardar la base norespondieron actualizada
norespondieron.to_csv('C:/Users/User/Downloads/Big data - TP2/TP3/norespondieron_actualizado.csv', index=False)

# Cargar la base de hogares para calcular la tasa de pobreza
datos_hogar = pd.read_excel('C:/Users/User/Downloads/Big data - TP2/TP3/usu_hogar_T423.xlsx')

# Filtrar datos para GBA (REGION == 1)
gba_hogar = datos_hogar[datos_hogar['REGION'] == 1].copy()

# Unir los datos de hogares con los datos de respondieron
hogares_completos = pd.merge(gba_hogar, respondieron, on=['CODUSU', 'NRO_HOGAR'], how='left')

# Calcular la tasa de pobreza ponderada utilizando PONDIH
hogares_completos['es_pobre'] = hogares_completos['pobre'].fillna(0)

# Calcular la tasa de pobreza ponderada
total_hogares_ponderado = hogares_completos['PONDIH'].sum()
total_hogares_pobres_ponderado = hogares_completos[hogares_completos['es_pobre'] == 1]['PONDIH'].sum()
tasa_pobreza_ponderada = total_hogares_pobres_ponderado / total_hogares_ponderado

print(f"Tasa de pobreza ponderada: {tasa_pobreza_ponderada:.2%}")

# Guardar los resultados
hogares_completos.to_csv('C:/Users/User/Downloads/Big data - TP2/TP3/hogares_completos.csv', index=False)

                           CODUSU  NRO_HOGAR  CH04  CH06     Edad  \
0   TQRMNOTUUHJMLPCDEIIAD00801670          1     1    76  76 años   
1   TQRMNOTUUHJMLPCDEIIAD00801670          1     2    79  79 años   
2   TQRMNOPUUHJKLQCDEIIAD00793187          1     1    65  65 años   
3   TQRMNOPUUHJKLQCDEIIAD00793187          1     2    66  66 años   
4   TQRMNOPUTHKLMNCDEIIAD00791268          1     2    49  49 años   
5   TQRMNOPUTHKLMNCDEIIAD00791268          1     1    20  20 años   
6   TQRMNOPUTHKLMNCDEIIAD00791268          1     1    17  17 años   
7   TQRMNORXPHKLMNCDEIIAD00791273          2     1    39  39 años   
8   TQRMNORXPHKLMNCDEIIAD00791273          2     1     5   5 años   
9   TQRMNOSPTHKKMPCDEIIAD00791441          1     1    89  89 años   
10  TQRMNOSPTHKKMPCDEIIAD00791441          1     1    72  72 años   
11  TQRMNOQWYHLMKRCDEIJAH00854987          1     1    57  57 años   
12  TQRMNOQWYHLMKRCDEIJAH00854987          1     2    51  51 años   
13  TQRMNOPWXHJMLQCDEIJAH00794277 

KeyError: 'PONDIH'

In [10]:
#PARTE 2
#INCISO 1
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score
import seaborn as sns

def evalua_metodo(modelo, X_train, y_train, X_test, y_test):
    # Ajustar el modelo con los datos de entrenamiento
    modelo.fit(X_train, y_train)
    
    # Predecir con los datos de prueba
    y_pred = modelo.predict(X_test)
    y_prob = modelo.predict_proba(X_test)[:, 1]  # Probabilidades de la clase positiva
    
    # Calcular la matriz de confusión
    matriz_confusion = confusion_matrix(y_test, y_pred)
    
    # Calcular las curvas ROC y AUC
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    auc_score = auc(fpr, tpr)
    
    # Calcular el accuracy score
    accuracy = accuracy_score(y_test, y_pred)
    
    # Graficar la matriz de confusión
    plt.figure(figsize=(8, 6))
    sns.heatmap(matriz_confusion, annot=True, fmt='d', cmap='Blues')
    plt.title('Matriz de Confusión')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    # Graficar la curva ROC
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {auc_score:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()
    
    # Devolver las métricas evaluadas
    metricas = {
        'matriz_confusion': matriz_confusion,
        'auc_score': auc_score,
        'accuracy': accuracy
    }
    
    return metricas

In [11]:
#INCISO 2:
from sklearn.model_selection import KFold
import numpy as np

def cross_validation(modelo, k, X, y):
    # Inicializar KFold con k particiones
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
    # Inicializar listas para almacenar las métricas de cada partición
    accuracies = []
    aucs = []
    
    # Iterar sobre cada partición generada por KFold
    for train_index, test_index in kf.split(X):
        # Crear conjuntos de entrenamiento y prueba para la partición actual
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Evaluar el modelo con la partición actual usando evalua_metodo
        metricas = evalua_metodo(modelo, X_train, y_train, X_test, y_test)
        
        # Almacenar las métricas de la partición actual
        accuracies.append(metricas['accuracy'])
        aucs.append(metricas['auc_score'])
    
    # Calcular las métricas promedio y desviación estándar
    accuracy_mean = np.mean(accuracies)
    accuracy_std = np.std(accuracies)
    auc_mean = np.mean(aucs)
    auc_std = np.std(aucs)
    
    # Devolver las métricas evaluadas
    metricas_cv = {
        'accuracy_mean': accuracy_mean,
        'accuracy_std': accuracy_std,
        'auc_mean': auc_mean,
        'auc_std': auc_std
    }
    
    return metricas_cv

In [12]:
#INCISO 3:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

def cross_validation(modelo, k, X, y):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
    errors = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        
        error = mean_squared_error(y_test, y_pred)
        errors.append(error)
    
    return np.mean(errors)

def evalua_config(modelo, configs, X, y, k=5):
    best_config = None
    best_error = float('inf')
    
    for config in configs:
        modelo.set_params(**config)
        error = cross_validation(modelo, k, X, y)
        
        if error < best_error:
            best_error = error
            best_config = config
    
    return best_config

In [13]:
#INCISO 4:
def evalua_metodo(modelo, X_train, y_train, X_test, y_test):
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    y_prob = modelo.predict_proba(X_test)[:, 1]
    
    matriz_confusion = confusion_matrix(y_test, y_pred)
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    auc_score = auc(fpr, tpr)
    accuracy = accuracy_score(y_test, y_pred)
    
    metricas = {
        'accuracy': accuracy,
        'roc_auc': auc_score
    }
    
    return metricas

def cross_validation(modelo, k, X, y):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
    errors = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        
        error = mean_squared_error(y_test, y_pred)
        errors.append(error)
    
    return np.mean(errors)

def evalua_config(modelo, configs, X, y, k=5):
    best_config = None
    best_error = float('inf')
    
    for config in configs:
        modelo.set_params(**config)
        error = cross_validation(modelo, k, X, y)
        
        if error < best_error:
            best_error = error
            best_config = config
    
    return best_config

def evalua_multiples_metodos(X, y, lambdas, k=5):
    resultados = []

    for lmbda in lambdas:
        # Configuraciones para la regresión logística
        configs_logistic = [
            {'penalty': 'l1', 'C': lmbda, 'solver': 'liblinear'},
            {'penalty': 'l2', 'C': lmbda}
        ]
        modelo_logistic = LogisticRegression()
        best_config_logistic = evalua_config(modelo_logistic, configs_logistic, X, y, k)

        if best_config_logistic is not None:
            modelo_logistic.set_params(**best_config_logistic)
            metricas_logistic = evalua_metodo(modelo_logistic, X_train, y_train, X_test, y_test)
            resultados.append({
                'model': 'Logistic Regression',
                'config': best_config_logistic,
                'lambda': lmbda,
                'accuracy': metricas_logistic['accuracy'],
                'roc_auc': metricas_logistic['roc_auc']
            })
        else:
            print("No se encontró una configuración óptima para la regresión logística.")
            
   # Modelo de Análisis de Discriminante Lineal
    modelo_lda = LinearDiscriminantAnalysis()
    metricas_lda = evalua_metodo(modelo_lda, X_train, y_train, X_test, y_test)
    resultados.append({
        'model': 'Linear Discriminant Analysis',
        'config': 'Default',
        'lambda': 'N/A',
        'accuracy': metricas_lda['accuracy'],
        'roc_auc': metricas_lda['roc_auc']
    })
    
    # Modelo de K-Vecinos más Cercanos (K = 3)
    modelo_knn = KNeighborsClassifier(n_neighbors=3)
    metricas_knn = evalua_metodo(modelo_knn, X_train, y_train, X_test, y_test)
    resultados.append({
        'model': 'K-Nearest Neighbors',
        'config': {'n_neighbors': 3},
        'lambda': 'N/A',
        'accuracy': metricas_knn['accuracy'],
        'roc_auc': metricas_knn['roc_auc']
    })
    
    # Crear un DataFrame con los resultados
    df_resultados = pd.DataFrame(resultados)
    
    return df_resultados
    


In [14]:
#PARTE 3:
#INCISO 1:
import pandas as pd

# Cargar los datasets con la opción low_memory=False
no_respondieron_df = pd.read_csv('C:/Users/User/Downloads/Big data - TP2/TP3/norespondieron.csv', low_memory=False)
respondieron_df = pd.read_csv('C:/Users/User/Downloads/Big data - TP2/TP3/respondieron_actualizado.csv', low_memory=False)

# Columnas a eliminar en cada dataset
columns_to_drop_no_respondieron = ['PP02C1', 'PP02C2', 'PP02C3', 'PP02C4', 'PP02C5', 'PP02C6', 'PP02C7', 'PP02C8', 'PP02E', 'PP02H', 'PP02I', 'adulto_equiv', 'ad_equiv_hogar']
columns_to_drop_respondieron = ['PP02C1', 'PP02C2', 'PP02C3', 'PP02C4', 'PP02C5', 'PP02C6', 'PP02C7', 'PP02C8', 'PP02E', 'PP02H', 'PP02I', 'adulto_equiv', 'ad_equiv_hogar', 'ingreso_necesario']

# Eliminar las columnas
no_respondieron_df.drop(columns=columns_to_drop_no_respondieron, inplace=True)
respondieron_df.drop(columns=columns_to_drop_respondieron, inplace=True)

# Mostrar las primeras filas de los dataframes modificados
print("No Respondieron DataFrame:")
print(no_respondieron_df.head())

print("\nRespondieron DataFrame:")
print(respondieron_df.head())

# Guardar los dataframes modificados en nuevos archivos CSV
no_respondieron_df.to_csv('C:/Users/User/Downloads/Big data - TP2/TP3/norespondieron_modified.csv', index=False)
respondieron_df.to_csv('C:/Users/User/Downloads/Big data - TP2/TP3/respondieron_actualizado_modified.csv', index=False)




No Respondieron DataFrame:
                          CODUSU  ANO4  TRIMESTRE  NRO_HOGAR  COMPONENTE  H15  \
0  TQRMNOPUUHJKLQCDEIIAD00793187  2023          4          1           1    2   
1  TQRMNOPUUHJKLQCDEIIAD00793187  2023          4          1           2    1   
2  TQRMNOPRWHKMLMCDEIJAH00794278  2023          4          1           2    1   
3  TQRMNOPRWHKMLMCDEIJAH00794278  2023          4          1           3    1   
4  TQRMNOPRWHKMLMCDEIJAH00794278  2023          4          1           4    1   

   REGION MAS_500  AGLOMERADO  PONDERA  ...  ADECIFR  IPCF DECCFR  IDECCFR  \
0       1       S          32      609  ...       12   0.0     12      NaN   
1       1       S          32      609  ...       12   0.0     12      NaN   
2       1       S          33     2254  ...       12   0.0     12      NaN   
3       1       S          33     2254  ...       12   0.0     12      NaN   
4       1       S          33     2254  ...       12   0.0     12      NaN   

   RDECCFR  GDECC

In [15]:
#Continuación del INCISO 1:
import pandas as pd
import numpy as np

# Cargar el dataset 
df = pd.read_csv('C:/Users/User/Downloads/Big data - TP2/TP3/respondieron_actualizado.csv', low_memory=False)

# Mostrar las columnas presentes en el DataFrame
print("Columnas presentes en el DataFrame:")
print(df.columns)

# Definir la variable dependiente (vector y)
y = df['pobre'].values

# Columnas que se deben eliminar de la matriz de variables independientes
columns_to_drop_from_X = ['pobre', 'CODUSU', 'MAS_500_x', 'MAS_500_y', 'CH05', 'PP09A_ESP']

# Eliminar solo las columnas que existen en el DataFrame
columns_to_drop = [col for col in columns_to_drop_from_X if col in df.columns]

# Definir las variables independientes (matriz X) eliminando las columnas especificadas
X = df.drop(columns=columns_to_drop).values

# Agregar una columna de unos a la matriz X si es necesario (para el término de intercepto en un modelo lineal)
X = np.hstack((np.ones((X.shape[0], 1)), X))

# Verificar las dimensiones de X e y
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)



Columnas presentes en el DataFrame:
Index(['CODUSU', 'ANO4', 'TRIMESTRE', 'NRO_HOGAR', 'COMPONENTE', 'H15',
       'REGION', 'MAS_500', 'AGLOMERADO', 'PONDERA',
       ...
       'RDECCFR', 'GDECCFR', 'PDECCFR', 'ADECCFR', 'PONDIH', 'Edad',
       'adulto_equiv', 'ad_equiv_hogar', 'ingreso_necesario', 'pobre'],
      dtype='object', length=182)
Shape of X: (4401, 179)
Shape of y: (4401,)


In [16]:
#INCISO 2:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.impute import SimpleImputer

# Cargar el dataset
df = pd.read_csv('C:/Users/User/Downloads/Big data - TP2/TP3/respondieron_actualizado.csv', low_memory=False)

# Definir la variable dependiente (vector y)
y = df['pobre'].values

# Columnas que se deben eliminar de la matriz de variables independientes
columns_to_drop_from_X = ['pobre', 'CODUSU', 'MAS_500_x', 'MAS_500_y', 'CH05', 'PP09A_ESP']

# Eliminar solo las columnas que existen en el DataFrame
columns_to_drop = [col for col in columns_to_drop_from_X if col in df.columns]

# Definir las variables independientes (matriz X) eliminando las columnas especificadas
X = df.drop(columns=columns_to_drop)

# Asegurarse de que todas las columnas en X sean numéricas
X = X.apply(pd.to_numeric, errors='coerce')

# Usar SimpleImputer para reemplazar valores NaN con la media de la columna
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Verificar si quedan NaN en X
print("Hay NaN en X:", np.isnan(X).any())

# Agregar una columna de unos a la matriz X si es necesario (para el término de intercepto en un modelo lineal)
X = np.hstack((np.ones((X.shape[0], 1)), X))

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def evalua_metodo(modelo, X_train, y_train, X_test, y_test):
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    y_prob = modelo.predict_proba(X_test)[:, 1]
    
    matriz_confusion = confusion_matrix(y_test, y_pred)
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    auc_score = auc(fpr, tpr)
    accuracy = accuracy_score(y_test, y_pred)
    
    metricas = {
        'accuracy': accuracy,
        'roc_auc': auc_score
    }
    
    return metricas

def cross_validation(modelo, k, X, y):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
    errors = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        
        error = mean_squared_error(y_test, y_pred)
        errors.append(error)
    
    return np.mean(errors)

def evalua_config(modelo, configs, X, y, k=5):
    best_config = None
    best_error = float('inf')
    
    for config in configs:
        modelo.set_params(**config)
        error = cross_validation(modelo, k, X, y)
        
        if error < best_error:
            best_error = error
            best_config = config
    
    return best_config

def evalua_multiples_metodos(X, y, lambdas, k=5):
    resultados = []

    for lmbda in lambdas:
        # Configuraciones para la regresión logística
        configs_logistic = [
            {'penalty': 'l1', 'C': lmbda, 'solver': 'liblinear'},
            {'penalty': 'l2', 'C': lmbda}
        ]
        modelo_logistic = LogisticRegression()
        best_config_logistic = evalua_config(modelo_logistic, configs_logistic, X, y, k)

        if best_config_logistic is not None:
            modelo_logistic.set_params(**best_config_logistic)
            metricas_logistic = evalua_metodo(modelo_logistic, X_train, y_train, X_test, y_test)
            resultados.append({
                'model': 'Logistic Regression',
                'config': best_config_logistic,
                'lambda': lmbda,
                'accuracy': metricas_logistic['accuracy'],
                'roc_auc': metricas_logistic['roc_auc']
            })
        else:
            print("No se encontró una configuración óptima para la regresión logística.")
    
    # Modelo de Análisis de Discriminante Lineal
    modelo_lda = LinearDiscriminantAnalysis()
    metricas_lda = evalua_metodo(modelo_lda, X_train, y_train, X_test, y_test)
    resultados.append({
        'model': 'Linear Discriminant Analysis',
        'config': 'Default',
        'lambda': 'N/A',
        'accuracy': metricas_lda['accuracy'],
        'roc_auc': metricas_lda['roc_auc']
    })
    
    # Modelo de K-Vecinos más Cercanos (K = 3)
    modelo_knn = KNeighborsClassifier(n_neighbors=3)
    metricas_knn = evalua_metodo(modelo_knn, X_train, y_train, X_test, y_test)
    resultados.append({
        'model': 'K-Nearest Neighbors',
        'config': {'n_neighbors': 3},
        'lambda': 'N/A',
        'accuracy': metricas_knn['accuracy'],
        'roc_auc': metricas_knn['roc_auc']
    })
    
    # Crear un DataFrame con los resultados
    df_resultados = pd.DataFrame(resultados)
    
    return df_resultados

# Ejecutar la función con la base respondieron
lambdas = [0.2, 2, 8]
resultados = evalua_multiples_metodos(X, y, lambdas, k=5)
print(resultados)


Hay NaN en X: False


C:\Users\User\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


                          model                       config lambda  accuracy  \
0           Logistic Regression  {'penalty': 'l2', 'C': 0.2}    0.2  0.997730   
1           Logistic Regression    {'penalty': 'l2', 'C': 2}      2  0.997730   
2           Logistic Regression    {'penalty': 'l2', 'C': 8}      8  0.997730   
3  Linear Discriminant Analysis                      Default    N/A  0.961407   
4           K-Nearest Neighbors           {'n_neighbors': 3}    N/A  0.970488   

    roc_auc  
0  1.000000  
1  1.000000  
2  1.000000  
3  0.993790  
4  0.992707  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score
from sklearn.impute import SimpleImputer

# Cargar el dataset
df = pd.read_csv('C:/Users/User/Downloads/Big data - TP2/TP3/respondieron_actualizado.csv', low_memory=False)

# Definir la variable dependiente (vector y)
y = df['pobre'].values

# Columnas que se deben eliminar de la matriz de variables independientes
columns_to_drop_from_X = ['pobre', 'CODUSU', 'MAS_500_x', 'MAS_500_y', 'CH05', 'PP09A_ESP']

# Eliminar solo las columnas que existen en el DataFrame
columns_to_drop = [col for col in columns_to_drop_from_X if col in df.columns]

# Definir las variables independientes (matriz X) eliminando las columnas especificadas
X = df.drop(columns=columns_to_drop)

# Asegurarse de que todas las columnas en X sean numéricas
X = X.apply(pd.to_numeric, errors='coerce')

# Usar SimpleImputer para reemplazar valores NaN con la media de la columna
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Verificar si quedan NaN en X
print("Hay NaN en X:", np.isnan(X).any())

# Definir la lista de valores de lambda a probar
lambdas = [10**n for n in range(-5, 6)]

# Crear modelos de regresión logística para Ridge y LASSO
modelo_ridge = LogisticRegression(penalty='l2', solver='liblinear', max_iter=10000)
modelo_lasso = LogisticRegression(penalty='l1', solver='liblinear', max_iter=10000)

# Función para realizar validación cruzada y obtener el error
def obtener_errores_cv(modelo, X, y, lambdas, k=10):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    errores_promedio = []
    proporciones_ceros = []

    for lmbda in lambdas:
        modelo.set_params(C=lmbda)
        errores = []
        ceros = []

        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            modelo.fit(X_train, y_train)
            y_pred = modelo.predict(X_test)
            error = mean_squared_error(y_test, y_pred)
            errores.append(error)

            if hasattr(modelo, 'coef_'):
                ceros.append(np.mean(modelo.coef_ == 0))

        errores_promedio.append(errores)
        proporciones_ceros.append(ceros)

    return errores_promedio, proporciones_ceros

# Obtener errores de validación cruzada
errores_ridge, _ = obtener_errores_cv(modelo_ridge, X, y, lambdas)
errores_lasso, ceros_lasso = obtener_errores_cv(modelo_lasso, X, y, lambdas)

# Convertir los errores en DataFrame para box-plots
df_errores_ridge = pd.DataFrame(errores_ridge, index=[f"10^{n}" for n in range(-5, 6)]).T
df_errores_lasso = pd.DataFrame(errores_lasso, index=[f"10^{n}" for n in range(-5, 6)]).T
df_ceros_lasso = pd.DataFrame(ceros_lasso, index=[f"10^{n}" for n in range(-5, 6)]).T

# Graficar los box-plots
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
sns.boxplot(data=df_errores_ridge)
plt.title('Ridge Regression: Prediction Error Distribution by Lambda')
plt.xlabel('Lambda')
plt.ylabel('Mean Squared Error')

plt.subplot(1, 2, 2)
sns.boxplot(data=df_errores_lasso)
plt.title('LASSO Regression: Prediction Error Distribution by Lambda')
plt.xlabel('Lambda')
plt.ylabel('Mean Squared Error')

plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
sns.boxplot(data=df_ceros_lasso)
plt.title('LASSO Regression: Proportion of Zero Coefficients by Lambda')
plt.xlabel('Lambda')
plt.ylabel('Proportion of Zero Coefficients')
plt.tight_layout()
plt.show()

# Guardar resultados
df_errores_ridge.to_csv('ridge_errors.csv', index=False)
df_errores_lasso.to_csv('lasso_errors.csv', index=False)
df_ceros_lasso.to_csv('lasso_zeros.csv', index=False)


In [ ]:
#INCISO 6:
# Cargar resultados de errores y proporción de ceros
df_errores_lasso = pd.read_csv('lasso_errors.csv')
df_ceros_lasso = pd.read_csv('lasso_zeros.csv')

# Seleccionar el mejor lambda para LASSO basado en el menor error
mean_errores_lasso = df_errores_lasso.mean()
best_lambda_lasso = lambdas[mean_errores_lasso.argmin()]

print(f"Mejor lambda para LASSO: {best_lambda_lasso}")

# Identificar variables descartadas por LASSO con el mejor lambda
modelo_lasso = LogisticRegression(penalty='l1', solver='liblinear', max_iter=10000, C=best_lambda_lasso)
modelo_lasso.fit(X, y)
variables_descartadas = np.where(modelo_lasso.coef_ == 0)[1]
nombres_variables_descartadas = df.drop(columns=columns_to_drop).columns[variables_descartadas]

print("Variables descartadas por LASSO con mejor lambda:")
print(nombres_variables_descartadas)

In [ ]:
#INCISO 7:
# Cargar resultados de errores
df_errores_ridge = pd.read_csv('ridge_errors.csv')
df_errores_lasso = pd.read_csv('lasso_errors.csv')

# Seleccionar el mejor lambda para Ridge y LASSO basado en el menor error
mean_errores_ridge = df_errores_ridge.mean()
mean_errores_lasso = df_errores_lasso.mean()

best_lambda_ridge = lambdas[mean_errores_ridge.argmin()]
best_lambda_lasso = lambdas[mean_errores_lasso.argmin()]

print(f"Mejor lambda para Ridge: {best_lambda_ridge}")
print(f"Mejor lambda para LASSO: {best_lambda_lasso}")

# Calcular el error cuadrático medio para los mejores modelos
modelo_ridge = LogisticRegression(penalty='l2', solver='liblinear', max_iter=10000, C=best_lambda_ridge)
modelo_lasso = LogisticRegression(penalty='l1', solver='liblinear', max_iter=10000, C=best_lambda_lasso)

kf = KFold(n_splits=10, shuffle=True, random_state=42)

ridge_errors = []
lasso_errors = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    modelo_ridge.fit(X_train, y_train)
    y_pred_ridge = modelo_ridge.predict(X_test)
    ridge_errors.append(mean_squared_error(y_test, y_pred_ridge))
    
    modelo_lasso.fit(X_train, y_train)
    y_pred_lasso = modelo_lasso.predict(X_test)
    lasso_errors.append(mean_squared_error(y_test, y_pred_lasso))

print(f"Error cuadrático medio (ECM) para Ridge: {np.mean(ridge_errors)}")
print(f"Error cuadrático medio (ECM) para LASSO: {np.mean(lasso_errors)}")

# Comparación de métodos de regularización
if np.mean(ridge_errors) < np.mean(lasso_errors):
    print("El método de regularización Ridge funciona mejor que LASSO basado en ECM.")
else:
    print("El método de regularización LASSO funciona mejor que Ridge basado en ECM.")

In [ ]:
#INCISO 9:
# Cargar el dataset modificado
no_respondieron_df = pd.read_csv('C:/Users/User/Downloads/Big data - TP2/TP3/norespondieron_modified.csv', low_memory=False)

# Preprocesamiento similar a la base `respondieron`
X_norespondieron = no_respondieron_df.apply(pd.to_numeric, errors='coerce')
X_norespondieron = imputer.transform(X_norespondieron)
X_norespondieron = np.hstack((np.ones((X_norespondieron.shape[0], 1)), X_norespondieron))

# Predecir la pobreza con el mejor modelo de LASSO
y_pred_norespondieron = modelo_lasso.predict(X_norespondieron)

# Proporción de hogares pobres en la submuestra norespondieron
proporcion_pobres = np.mean(y_pred_norespondieron)
print(f"Proporción de hogares pobres en la submuestra norespondieron: {proporcion_pobres:.2%}")
